In [48]:
import re
from zipfile import ZipFile

import petl as etl
from pipeline_utils.filesystem.paths import RAW_DATA, REF_DATA

In [49]:
onspd_zip = RAW_DATA / 'onspd.zip'

In [50]:
postcode_areas = ['NE', 'TD', 'DL', 'DH', 'SR', 'CA']

search = re.compile(f'Data/multi_csv/.*({'|'.join(postcode_areas)})\\.csv')

files = [
    f for f in
    ZipFile(onspd_zip).namelist()
    if
    re.match(search, f)
]

In [51]:
postcodes = etl.cat(
    *[etl.fromcsv(etl.ZipSource(onspd_zip, file)) for file in files]
).selecteq(
    'doterm', ''
).cut(
    *'pcds oslaua lat long oa21 lsoa21 msoa21'.split(' ')
)

In [52]:
postcodes.tocsv(REF_DATA / 'onspd_extract.csv')